# Importing packages

In [2]:
from AugmentedSocialScientist import swedish_bert

In [30]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

# Loading Data

In [5]:
df = pd.read_csv('./swedish-sentiment.csv')

In [7]:
train, test = train_test_split(df, test_size=0.2)

# Encoding

In [13]:
train_dataloader = swedish_bert.encode(train.text.values, train.value.values, batch_size=32)

In [14]:
test_dataloader = swedish_bert.encode(test.text.values, test.value.values, batch_size=256)

# Training (and Saving the Trained Model)

In [16]:
score = swedish_bert.run_training(train_dataloader, 
                                  test_dataloader, 
                                  n_epochs=3, 
                                  lr=5e-5, 
                                  seed_val=2018, 
                                  save_model_as="swedish_sentiment")

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedi


======== Epoch 1 / 3 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:00:16

Running Validation...

  Average test loss: 0.65
  Validation took: 0:00:02
              precision    recall  f1-score   support

           0       0.71      0.68      0.69        78
           1       0.54      0.57      0.55        51

    accuracy                           0.64       129
   macro avg       0.62      0.62      0.62       129
weighted avg       0.64      0.64      0.64       129


======== Epoch 2 / 3 ========
Training...

  Average training loss: 0.52
  Training epcoh took: 0:00:16

Running Validation...

  Average test loss: 0.66
  Validation took: 0:00:01
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        78
           1       0.58      0.67      0.62        51

    accuracy                           0.67       129
   macro avg       0.67      0.67      0.67       129
weighted avg       0.69      0.67    

# Using the local model to predict

In [26]:
predict_loader = swedish_bert.encode(test.text.values)

In [27]:
pred = swedish_bert.predict_with_model(predict_loader, model_path='./models/swedish_sentiment/', proba=True)

In [32]:
test['pred'] = np.argmax(pred, axis=1)

/pbs/throng/humanum/anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
test['prob_1'] = pred[:,1]

/pbs/throng/humanum/anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
test

,text,value,pred,prob_1
95,"Jobb för en FLYKTING?? Hej,. Jag har blivit go...",1,1,0.744390
518,20% av sveriges befolkning har invandrat. 20% ...,1,1,0.950754
255,"Dåliga nyheter i SVD: ""EU kan stoppa återgång ...",1,1,0.731366
200,Detta behövs för att hålla samma nivå på läkar...,0,0,0.307097
566,14-åring utvisas till Italien. 14-åringen Shar...,1,0,0.277097
...,...,...,...,...
123,Ny flyktingrutt etabklerad via Svarta Havet. N...,1,1,0.528245
58,"Får asylsökande DNA testas? Ja, hur ligger det...",1,1,0.577488
424,Är det dags att dela Sverige i två nationer? P...,0,1,0.605603
180,"V4-länderna (Polen, Tjeckien, Slovakien, Unger...",0,0,0.428451


In [38]:
test.to_csv('prediction.csv', index=False)